In [1]:
import torch
from bilstm import BiLSTM
from load_data import num_words, num_tags, words, tags, word2idx, tag2idx, train_X, train_y, test_X, test_y, train_size

In [2]:
new_model = BiLSTM(num_words, tag2idx, embedding_dim=50, hidden_dim=200)
new_model.load_state_dict(torch.load("./weights/bilstm.pth", map_location='cpu'))

C:\Users\ziyet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\nn\modules\rnn.py:67: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


<All keys matched successfully>

In [3]:
# Calculate Precision, Recall, and F1 Score
new_model.eval()
TP = 0
TN = 0
FP = 0
FN = 0
for i in range(len(test_X)):
    cur_X = torch.tensor(test_X[i:i+1], dtype = torch.long)
    cur_y = torch.tensor(test_y[i], dtype = torch.long)

    pred = torch.argmax(new_model(cur_X)[0], dim=1)
    TP += torch.sum((pred!=tag2idx['O']) * (pred == cur_y))
    TN += torch.sum((pred==tag2idx['O']) * (pred == cur_y))
    FP += torch.sum((pred!=tag2idx['O']) * (pred != cur_y))
    FN += torch.sum((pred==tag2idx['O']) * (pred != cur_y))
    
    if (i+1)%1000 == 0:
        print(i)

prec = TP/(TP+FP)
recall = TP/(TP+FN)
f1 = prec*recall/(prec+recall)
print("Precision: ",prec)
print("Recall: ", recall)
print("F1 Score: ", f1*2)

999


KeyboardInterrupt: 

In [4]:
# Inference on test set
new_model.eval()
for i in range(7990, 8000):
    cur_X = torch.tensor(test_X[i:i+1])
    cur_y = torch.tensor(test_y[i:i+1])

    pred = torch.argmax(new_model(cur_X),dim=2)
    pred_tag = ""
    t_tag = ""
    sentence = ""
    for j in range(0, len(cur_X[0])):
        sentence += words[cur_X[0][j]]+" "
        t_tag += tags[cur_y[0][j]] + " "
        pred_tag += tags[pred[0][j]] + " "
    
    print(sentence,'\n')
    print(t_tag,'\n')
    print(pred_tag,'\n\n')

The famine killed as many as 10 million people in what was then Soviet Ukraine .  

O O O O O O O O O O O O O O B-geo O  

O O O O O O O O O O O O O O B-geo O  


Many analysts say the famine was not from natural causes , but instead was the result of government policies under Soviet dictator Josef Stalin .  

O O O O O O O O O O O O O O O O O O O O O O B-per I-per O  

O O O O O O O O O O O O O O O O O O O O O O B-per I-per O  


Ukrainian President Viktor Yushchenko supported the bill .  

B-gpe B-per I-per I-per O O O O  

B-gpe B-per I-per I-per O O O O  


Most have been released unharmed , often after a payment of ransom .  

O O O O O O O O O O O O O  

O O O O O O O O O O O O O  


On Saturday , he spoke in the capital , Kiev , to mark the 73rd anniversary of the tragedy , known in Ukraine as the Holodomor .  

O B-tim O O O O O O O B-per O O O O O O O O O O O O B-geo O O B-geo O  

O B-tim O O O O O O O B-org O O O O B-tim O O O O O O O B-geo O O O O  


China denies it is fai